# Hi Split + Coarsening
Hi splitting for bigger datasets

Running the Hi Splitter on large datasets can be extremely time-consuming. In this tutorial, we'll explore methods to expedite the process without compromising the quality of the splits.


In [1]:
import pandas as pd
import numpy as np

import lohi_splitter as lohi

## Data Loading
In this tutorial, we will partition a dataset containing 6,000 molecules.

In [2]:
drd2_hi = pd.read_csv('data/drd2_hi.csv', index_col=0)
drd2_hi

,smiles,value
0,Brc1ccc(-[n+]2cc[n+](Cc3ccccc3)cc2)c2cc[nH]c12,True
1,Brc1ccc(CNCCN2CCN(Cc3cc4ccccc4[nH]3)CC2)cc1,False
2,Brc1ccc(N2CCN(Cc3ccccc3)CC2)c2cc[nH]c12,True
3,Brc1ccc(NCCN2CCN(CCc3c[nH]c4ccccc34)CC2)cc1,True
4,Brc1ccc(NCCN2CCN(Cc3cc4ccccc4[nH]3)CC2)cc1,True
...,...,...
6262,c1cnc(N2CCN(CCCOc3ccc(-c4nc5ccccc5o4)cc3)CC2)nc1,True
6263,c1cnc(N2CCN(CCCSc3nc4ccccc4s3)CC2)nc1,True
6264,c1cnc(N2CCN(Cc3c[nH]c4ncccc34)CC2)nc1,False
6265,c1cncc(CN[C@H]2C3C4CC5C6C4CC3C6C52)c1,False


## Problem Formulation
The following code cell is time-intensive and has been turned off by default. If you decide to activate and run it, be aware that it may not respond to interruption commands. In such cases, you might need to restart the kernel.

In [3]:
# Change it to True
RUN_CELL = False

if RUN_CELL:
    smiles = drd2_hi['smiles'].to_list()
    partition = lohi.hi_train_test_split(smiles, similarity_threshold=0.4, 
                                        train_min_frac=0.7, test_min_frac=0.1)


## Graph Coarsening

In this section, we'll delve into the mechanics behind the Hi splitter. For a more in-depth understanding, refer to the tutorial `03_hi_under_the_hood.ipynb`.

The primary objectives of the Hi algorithm when partitioning a dataset into k parts are:
1) Ensuring that there aren't any pairs of molecules that reside in different partitions but are similar to each other.
2) Ensuring that the partitions adhere to specified size constraints (e.g., training set approximately 80%, test set approximately 20%).

To achieve this, the algorithm constructs a neighborhood graph. In this graph, each node corresponds to a molecule, and two nodes are linked if their respective molecules are similar. Typically, the resulting graph might have one giant connected component and several smaller ones. The smaller components can be allocated to different partitions without much hassle, so the primary challenge lies in splitting the giant connected component into k parts.

To enhance the algorithm's speed, we can structure the graph using clusters of molecules rather than individual ones. This process, known as coarsening, enables more efficient computations. Let's explore the differences between the standard and coarsened approaches.

In [4]:
# This is actually happening inside of the Hi splitter

# 1. Build a neighborhood graph. The outcome is a graph structure from the networkx library.
smiles = drd2_hi['smiles'].to_list()
neighborhood_graph = lohi.get_neighborhood_graph(smiles, threshold=0.4)

# 2. Segregate the graph into its largest connected component and the smaller, disconnected components.
giant_component, small_components = lohi.get_giant_component(neighborhood_graph)

# 3. This section isn't central to the Hi splitter's process but helps in relabeling 
# nodes of the giant component for clearer indexing. This can be useful for subsequent operations.
import networkx as nx
old_nodes_to_new = dict(zip(giant_component.nodes(), range(giant_component.number_of_nodes())))
giant_component = nx.relabel_nodes(giant_component, old_nodes_to_new)

Now, we'll experiment with various coarsening settings. We'll cluster groups of molecules together if their similarity exceeds the `coarsening_threshold`. Then, we'll observe how these settings influence the coarsened graph.

In [5]:
print('threshold num_nodes average_degree')
for coarsening_threshold in [1.0, 0.90, 0.80, 0.70, 0.60, 0.50, 0.40]:
    coarsed_giant_component, _ = lohi.coarse_graph(giant_component, coarsening_threshold)
    node_degrees = list(dict(coarsed_giant_component.degree()).values())
    print(coarsening_threshold, len(coarsed_giant_component), np.mean(node_degrees))

threshold num_nodes average_degree
1.0 6082 107.0121670503124
0.9 5160 86.87093023255814
0.8 3696 56.81547619047619
0.7 1963 34.751910341314314
0.6 1128 24.48049645390071
0.5 648 17.126543209876544
0.4 387 11.534883720930232


As observed, the lower the threshold, the more compact the coarsened graph becomes. For instance, clustering molecules with a similarity greater than 0.4 results in a graph with 20 times fewer nodes and an average degree reduced by 10 times.


## Faster Hi Splitter

Fortunately, you don't have to handle graph coarsening on your own. The Hi splitter is equipped to manage this for you. You simply need to provide the `coarsening_threshold` parameter. We'll delve into its execution shortly, but first, let's discuss another method to expedite the splitting process.

Internally, the Hi splitter employs an optimization routine that maximizes a score function. While it quickly identifies a non-optimal solution, the algorithm might spend significant time improving it. If achieving the absolute optimal solution isn't mandatory for your use case, you can leverage early stopping. This approach halts the optimization once the difference between the current score and the theoretical optimum is sufficiently small. This is determined using the `max_mip_gap` parameter. When the fraction (score - optimum) / optimum drops below the `max_mip_gap` value, the optimization ceases. By default, this parameter is set to 0.1. However, if you prefer a quicker yet possibly less precise solution, you can increase it to 0.3. If the algorithm ends up discarding too many molecules, you can gradually raise this threshold.

In the subsequent section, you'll find the code alongside a detailed breakdown of how these parameters influence the results.


In [6]:
smiles = drd2_hi['smiles'].to_list()

# Define a threshold for similarity. Molecules with similarity > 0.4 are considered similar.
similarity_threshold = 0.4

# Set fractions for the train and test sets.
# Increase their sum to discard fewer molecules. Decrease it to speed up computations.
train_min_frac = 0.70
test_min_frac = 0.10

# Threshold for graph clustering.
# Increase it to discard fewer molecules. Decrease it to speed up computations.
coarsening_threshold = 0.4

# How close we should be to the theoretical optimum to terminate the optimization.
# Should be in [0, 1].
# Decrease it to discard fewer molecules. Increase it to speed up computations.
max_mip_gap = 0.01

partition = lohi.hi_train_test_split(smiles=smiles,
                                     similarity_threshold=similarity_threshold,
                                     train_min_frac=train_min_frac,
                                     test_min_frac=test_min_frac,
                                     coarsening_threshold=coarsening_threshold,
                                     max_mip_gap=max_mip_gap)

Total molecules in the giant component: 6082
Min train size 4257
Min test size 608
Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 24 2021 

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 4079 (-774) rows, 774 (0) columns and 8928 (-1548) elements
Clp1000I sum of infeasibilities 1.46223e-06 - average 3.58477e-10, 0 fixed columns
Coin0506I Presolve 4079 (0) rows, 774 (0) columns and 8928 (0) elements
Clp0029I End of values pass after 774 iterations
Clp0014I Perturbing problem by 0.001% of 0.90766858 - largest nonzero change 2.99959e-05 ( 0.002488166%) - largest zero change 0
Clp0000I Optimal - objective value 6082
Clp0000I Optimal - objective value 6082
Clp0000I Optimal - objective value 6082
Coin0511I After Postsolve, objective 6082, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 6082 - 0 iterations time 0.132, Presolve 0.00, Idiot 0.12

Starting MIP optimization
Cgl0003I 34 fixed, 0 tightene

These settings should enable you to split the HIV dataset, comprising 40k molecules, in just a few hours. If this duration is still too lengthy for your dataset, or if the standard Hi split isn't entirely suitable for your task, refer to 03_hi_under_the_hood.ipynb for insights on how to modify the approach.